In [97]:
import numpy as np
# import matplotlib as plt
import PIL.Image as img
from pathlib import Path
import math
working_dir = Path().resolve()
print(working_dir)

/home/vexrinawlw/Документы/projects/neuron_labs


In [98]:
def convert(file_name):
    '''
    Перевод png файла в массив тюплов RGBA
    (red green blue alpha)
    '''
    image = img.open(working_dir/file_name)
    return image.convert('RGBA').getdata()

In [99]:
def take_datas(letters):
    '''
    Перевод всех изображений в матрицу и сохранений всех матриц в отдельный массив
    '''
    letters_data = []
    for letter in letters:
        for i in range(4): # 4 - количество образов буквы
            data = convert(f'letters/{letter}{i+1}.png')
            letters_data.append(data)
    return letters_data

In [100]:
letters = ['q','h','i','w'] # буквы, для которых составлены образы

datas = take_datas(letters)
# print(len(datas))
def tuples_to_mtx(data:list[tuple])->list[list[int]]:
    '''
    Конвентирует массив тюплов в матрицу, 
    где ТОЛЬКО черный цвет с alpha=255 - единица,
    а любое другое значение - 0
    '''
    result = [[]]
    i, j = 0, 0
    for item in data:
        if j == 32:
            result.append([])
            i+=1
            j=0
        if item[0] == 0 and item[1] == 0 and item[2] == 0 and item[3] == 255:
            result[i].append(1)
            j+=1
        else:
            result[i].append(0)
            j+=1
    return result


In [101]:
mtxs=[]
for data in datas:
    mtxs.append(tuples_to_mtx(data))

In [102]:
def print_mtx(mtx):
    for row in mtx:
        print(row)

def mtx_to_vector(mtx):
    result = []
    for row in mtx:
        for item in row:
            result.append(item)
    return result

In [103]:
class Neuron:
    def sigmoid(self, x: float):
        return 1 / (1 + math.exp(-x))

    def __init__(self, vectors, response, epoch:int):
        self.weight = np.random.uniform(low=-0.03, high=0.03, size=(1024,))
        self.letters = vectors
        self.epoch = epoch
    
    def weight_sum(self):
        letter = self.letters[0]
        self.after_multiply = []
        for index in range(1024):
            self.after_multiply.append(self.weight[index]*letter[index])

        self.sum = sum(self.after_multiply)
        self.activate = self.sigmoid(self.sum)

    # def train_epoch(self):
    #     k = 0
    #     for i in range(self.epoch):
    #         weight_sum()


In [104]:
vector = []
for i in range(0,16,4):
    vector.append(mtx_to_vector(mtxs[i]))
N = Neuron(vector, [1,2,3,4], 25)

import time

start = time.time()
N.weight_sum()
end = time.time()
print(end-start)
N.activate

0.0019068717956542969


0.471438731176727

In [105]:
arr = np.random.uniform(low=-0.03, high=0.03, size=(4,10))
print(len(arr))
print(len(arr[0]))

4
10
